In [1]:
! pip install xgboost
! pip install pandas
! pip install scikit-learn
! pip install pyarrow

In [2]:
################################ Libraries ################################
from Input_Variables.read_vars import xgb_reg_model_storage_location, xgb_class_model_storage_location, random_seed, \
                                      evaluation_metrics_output_storage, \
                                      feature_importance_storage_location, \
                                      overall_feature_importance_plot_location
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.scaling_pipeline import Feature_Transformations
from Model_Creation.pyspark_xgboost import Create_PySpark_XGBoost
from Model_Predictions.pyspark_model_preds import Model_Predictions
from Model_Evaluation.pyspark_model_eval import Evaluate_Model
from Feature_Importance.model_feature_importance import Feature_Importance
from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot
import os


################################ Read In Modules ################################
reading_data=Reading_Data()
feature_transformations=Feature_Transformations()
create_pyspark_xgboost=Create_PySpark_XGBoost()
model_predictions=Model_Predictions()
evaluate_model=Evaluate_Model()
feature_importance=Feature_Importance()
xgboost_classification_plot=XGBoost_Classification_Plot()


################################ Read In Data ################################
# Training Summary Stats Data
training_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_train'), x),os.listdir('/cephfs/summary_stats/all_train')))
training_files=[i for i in training_files if not ('.crc' in i or 'SUCCESS' in i)]


# Cross Validation Summary Stats Data
val_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_val'), x),os.listdir('/cephfs/summary_stats/all_val')))
val_files=[i for i in val_files if not ('.crc' in i or 'SUCCESS' in i)]


# Testing Summary Stats Data
test_files=list(map(lambda x: os.path.join(os.path.abspath('/cephfs/summary_stats/all_test'), x),os.listdir('/cephfs/summary_stats/all_test')))
test_files=[i for i in test_files if not ('.crc' in i or 'SUCCESS' in i)]

# Calling DataFrames
summary_stats_train=reading_data.read_in_all_summary_stats(file_list=training_files)
summary_stats_val=reading_data.read_in_all_summary_stats(file_list=val_files)
summary_stats_test=reading_data.read_in_all_summary_stats(file_list=test_files)


################################ Combine Train and Cross Validation ################################
df_train_val_combined=summary_stats_train.union(summary_stats_val)
df_train_val_combined.show(2)
print((df_train_val_combined.count(), len(df_train_val_combined.columns)))


################################ Stages: Scaling Using Custom Transformer ################################
pipeline_transformation_stages=feature_transformations.numerical_scaling(df=df_train_val_combined)

print(pipeline_transformation_stages)


################################ XGBoost Regression Model ################################
xgboost_regression_model=create_pyspark_xgboost\
        .initial_training_xgboost_regression(ml_df=summary_stats_train,
                                             stages=pipeline_transformation_stages, 
                                             random_seed=random_seed,
                                             xgb_reg_model_storage_location=xgb_reg_model_storage_location)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/30 00:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+-----+-----+-----------------+----------------+-------------+-------------+------------------+-----------------+------------------+------+----+-----+------------+--------------------+------------------+------------------+----------+----------+---------------+------------+------+-----------+-----------------+----------------+
|NumId|Chunk|ShortTermVariance|LongTermVariance|VarianceRatio|SampleEntropy|PermutationEntropy|             Mean|            StdDev|Median| Min|  Max|AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|DiffPrevious|target|Sex_Encoded|Treatment_Encoded|AgeGroup_Encoded|
+-----+-----+-----------------+----------------+-------------+-------------+------------------+-----------------+------------------+------+----+-----+------------+--------------------+-----------------

You enabled use_gpu in spark local mode. Please make sure your local node has at least 1 GPUs


23/05/30 00:32:06 WARN MemoryStore: Not enough space to cache rdd_53_72 in memory! (computed 3.8 MiB so far)
23/05/30 00:32:06 WARN BlockManager: Persisting block rdd_53_72 to disk instead.


23/05/30 00:32:06 WARN MemoryStore: Not enough space to cache rdd_53_75 in memory! (computed 3.8 MiB so far)
23/05/30 00:32:06 WARN BlockManager: Persisting block rdd_53_75 to disk instead.


...

23/05/30 00:35:31 WARN MemoryStore: Not enough space to cache rdd_101_198 in memory! (computed 3.1 MiB so far)


[00:35:31] WARNING: ../src/learner.cc:553: ===================> (195 + 4) / 200]
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[00:35:31] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[00:35:31] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS i

[234.416s][warning][gc,alloc] Executor task launch worker for task 39.0 in stage 80.0 (TID 3789): Retried waiting for GCLocker too often allocating 5839 words
23/05/30 00:35:32 ERROR Executor: Exception in task 39.0 in stage 80.0 (TID 3789)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2121)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2157)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream$FieldValues.<init>(ObjectInputStream.java:2606)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2457)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2257)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58036)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last)

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
xgboost_regression_model.stages[-1].avgMetrics

NameError: name 'xgboost_regression_model' is not defined

In [ ]:
xgboost_regression_model.stages[-1].subModels

In [ ]:
xgboost_regression_model.stages[-1].bestModel().

In [ ]:
from Model_Predictions.pyspark_model_preds import Model_Predictions
from Model_Evaluation.pyspark_model_eval import Evaluate_Model
from Feature_Importance.model_feature_importance import Feature_Importance
from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot
import os

In [ ]:
model_predictions=Model_Predictions()
evaluate_model=Evaluate_Model()
feature_importance=Feature_Importance()
xgboost_classification_plot=XGBoost_Classification_Plot()

In [ ]:
################################ Testing Predictions ################################
testing_predictions=model_predictions.create_predictions_with_model(test_df=summary_stats_test, 
                                                                    model=xgboost_regression_model)
testing_predictions.show(10)

In [ ]:
################################ Model Evaluation ################################
model_evaluation=evaluate_model.regression_evaluation(testing_predictions=testing_predictions, 
                                                      eval_csv_location=evaluation_metrics_output_storage)
model_evaluation.head()


In [ ]:
################################ Feature Importance ################################
feature_importance_df=feature_importance \
                            .feature_importance_accuracy_gain(xgboost_model=xgboost_regression_model, 
                                                              feature_importance_storage_location=feature_importance_storage_location)
feature_importance_df.head(10)

In [ ]:
################################ Feature Importance Plot ################################
overall_feature_plot=xgboost_classification_plot.feature_overall_importance_plot(feature_importance_df=feature_importance_df,
                                                                                 overall_importance_plot_location=overall_feature_importance_plot_location)